# Threat Model Glasswall ICAP Cloud Deployment
## Network Visualisation

In [1]:
import ipysheet
import ipywidgets
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Threat Model Workflow

In [3]:
%%html
<div id="mynetwork"></div>

In [4]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

var SHAPE = "circle", //"box", //"square"
    SHAPE2 = "box",
    BLUE = "#99CDFF",
    YELLOW = "#FEEBA7",
    GREEN = "#C6E7B0",
    PURPLE = "#DAADF0",
    ORANGE = "#FCC603",
    PINK = "#FFBBB2",
    BLACK = "#2B1B17";
    
require(['vis'], function(vis){    
    
    // create an array with nodes
    var nodes = new vis.DataSet([ 
      // Trust Boundary ID 
      { id: 1000, label: "Trust \nBoundary", color: BLUE, shape: SHAPE },  
      // Asset ID  
      { id: 1001, label: "Asset", color: YELLOW, shape: SHAPE },  
      // Threat Actor ID
      { id: 1002, label: "Threat \nActor", color: PINK, shape: SHAPE }, 
      // Security Control ID
      { id: 1003, label: "Security \nControl", color: GREEN, shape: SHAPE }, 
      // Vulnerability
      { id: 1004, label: "Vulnerability", color: PURPLE, shape: SHAPE2 },
      //Risk
      { id: 1005, label: "Risk", color: ORANGE, shape: SHAPE2 },          
    ]);

    // create an array with edges
    var edges = new vis.DataSet([
      {id: 47, from: 1000, to: 1002, label: "Threat", arrows: "to", color: BLACK },
      {id: 48, from: 1001, to: 1002, label: "Threat", arrows: "to", color: BLACK },
      {id: 49, from: 1002, to: 1004, arrows: "to", color: BLACK },
      {id: 50, from: 1004, to: 1005, arrows: "to", color: BLACK },
      {id: 51, from: 1005, to: 1003, label: "Risk Possibility \nRisk Impact \nRisk Level", arrows: "to", color: BLACK },
    ]);    

    // create a network
    var container = document.getElementById("mynetwork");
    var data = {
      nodes: nodes,
      edges: edges,
    };
    var options = {
      width: '900px',
      height: '400px',
      locale: 'en',
      physics: false,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
        hierarchical: {
          enabled: true,
          direction: 'LR',
          nodeSpacing: 200,
          levelSeparation: 250,  
          sortMethod: 'directed',
        },
      },    
    };    
    var network = new vis.Network(container, data, options);
});

<IPython.core.display.Javascript object>

## Threat Model Network Visualisations

First let's pull the data from this excelsheet

In [5]:
df = pd.read_excel (r'data/threat_table.xlsx')

threat_table = ipysheet.from_dataframe(df)
threat_table

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format=None, row_end=11, row_start=0, squee…

In [6]:
# save changes to excel
df_to_excel = ipysheet.to_dataframe(threat_table)
df_to_excel.to_excel('data/threat_table.xlsx', index=False)

In [7]:
# Reopen the edited file

threat_table = pd.read_excel (r'data/threat_table.xlsx')

In [8]:
nodes = []
edges = []

nodes_ids = {}
id_counter = 1
edge_id = 0
edges_pairs = []


def wrap_by_word(s, n):
    """
    Returns a string where \n is inserted between every n words
    
    :param s: string
    :param n: integer, number of words
    :return:
    """
    a = s.split()
    ret = ''
    for i in range(0, len(a), n):
        ret += ' '.join(a[i:i+n]) + '\n'
    return ret


def add_node(id_counter, label, group, n=3):
    if label not in nodes_ids:
        d = {}
        
        d["id"] = id_counter
        
        if isinstance(label, str):  
            d["label"] = wrap_by_word(label, n)
        else:
            d["label"] = label
        
        d["group"] = group
        
        if group == "TrustBoundary":
            d["color"] = "#99CDFF"
            d["shape"] = "circle"
            d["level"] = 0   
        elif group == "Asset":
            d["color"] = "#FEEBA7"
            d["shape"] = "circle"
            d["level"] = 0               
        elif group == "ThreatActor":
            d["color"] = "#FFBBB2"
            d["shape"] = "circle"
            d["level"] = 1              
        elif group == "SecurityControl":
            d["color"] = "#C6E7B0"
            d["shape"] = "circle"
            d["level"] = 4               
        elif group == "Vulnerability":
            d["color"] = "#DAADF0"
            d["shape"] = "box"
            d["level"] = 2               
        elif group == "Risk":
            d["color"] = "#FCC603"
            d["shape"] = "box"
            d["level"] = 3   
        
        nodes_ids[label] = id_counter
        nodes.append(d)
        id_counter += 1
        
    return id_counter


def add_edge(a, b, edge_id, label="", length=100):
        
    m = edges_pairs.count((a, b))
    
    if m == 0 :
        color = "black"
    if m == 1:
        color = "red"
    if m == 2:
        color = "blue"
        
    edges_pairs.append((a, b))
           
    edges.append({"id": edge_id, 
                "from": a, 
                "to": b, 
                "label": label, 
                "arrows": "to", 
                "color": color,
                "length": length 
                 })
    edge_id += 1
    
    return edge_id


for i in range(len(threat_table)):
    
    # NODES
    label1 = threat_table.iloc[i]['Trust_Boundary_Description']
    
    if threat_table.iloc[i]['Trust_Boundary_ID'][0]=='T':
        group = "TrustBoundary"
    elif threat_table.iloc[i]['Trust_Boundary_ID'][0]=='A':
        group = "Asset"
    
    id_counter = add_node(id_counter, label1, group, n=2)
    
    label2 = threat_table.iloc[i]['Threat_Actor_Description']
    id_counter = add_node(id_counter, label2, group="ThreatActor", n=2)
    
    label3 = threat_table.iloc[i]['Vulnerability']
    id_counter = add_node(id_counter, label3, group="Vulnerability")
    
    label4 = threat_table.iloc[i]['Risk']
    id_counter = add_node(id_counter, label4, group="Risk")
    
    label5 = threat_table.iloc[i]['Security_Control_Description']
    id_counter = add_node(id_counter, label5, group="SecurityControl", n=2)
    
    # EDGES
    label6 = wrap_by_word(threat_table.iloc[i]['Threat'], 2)
    label7 = "\nRisk Possibility: \n" +  threat_table.iloc[i]['Risk_Possibility'] + "\nRisk Impact: \n" + threat_table.iloc[i]['Risk_Impact'] + "\nRisk Level: \n" + threat_table.iloc[i]['Risk_Level']  
    
    edge_id = add_edge(nodes_ids[label1], nodes_ids[label2], edge_id, label6, length=200)
    edge_id = add_edge(nodes_ids[label2], nodes_ids[label3], edge_id)    
    edge_id = add_edge(nodes_ids[label3], nodes_ids[label4], edge_id)
    edge_id = add_edge(nodes_ids[label4], nodes_ids[label5], edge_id, label7, length=300)

    
threat_table_2 = {"nodes": nodes, "edges": edges}

# save to JSON
with open('threat_table_2.json', 'w') as fp:
    json.dump(threat_table_2, fp)

## Option 1: With Hierarchy

In [9]:
%%html
<div id="mynetwork2"></div>

In [10]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


require(['vis'], function(vis){
    
    var json = $.getJSON("threat_table_2.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
    
    var options = {
      width: '900px',
      height: '800px',
      locale: 'en',
      physics: false,
      interaction: {
        hover:true, 
        tooltipDelay: 300
      },        
      layout: {
        randomSeed: 1,
        improvedLayout: true,
        hierarchical: {
          enabled: true,
          direction: 'UD',
          nodeSpacing: 200,
          levelSeparation: 250,  
          sortMethod: 'directed',
        },
      },
    }; 
    
    var container = document.getElementById("mynetwork2");
    
});

<IPython.core.display.Javascript object>

## Option 2: No Hierarchy

In [11]:
%%html
<div id="mynetwork3"></div>

In [12]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});


var SHAPE = "circle", //"box", //"square"
    SHAPE2 = "box",
    BLUE = "#99CDFF",
    YELLOW = "#FEEBA7",
    GREEN = "#C6E7B0",
    PURPLE = "#DAADF0",
    ORANGE = "#FCC603",
    PINK = "#FFBBB2";
    

require(['vis'], function(vis){ 
    
    var json = $.getJSON("threat_table_2.json")
      .done(function(data){
        var data = {
          nodes: data.nodes,
          edges: data.edges
        };
        var network = new vis.Network(container, data, options);
      });
       
    var options = {      
      width: '900px',
      height: '800px',
      physics: true,
      groups: {
        TrustBoundary: {
          shape: SHAPE,
          color: BLUE,
        },
        Asset: {
          shape: SHAPE,
          color: YELLOW,
        },
        ThreatActor: {
          shape: SHAPE,
          color: PINK,
        },
        SecurityControl: {
          shape: SHAPE,
          color: GREEN,
        },
        Vulnerability: {
          shape: SHAPE2,
          color: PURPLE,
        },
        Risk: {
          shape: SHAPE2,
          color: ORANGE,
        },          
      },    
    };    
    
    var container = document.getElementById("mynetwork3");
});

<IPython.core.display.Javascript object>